# Usage

In [ ]:
from importlib import reload
import os

from obspy.clients.filesystem.sds import Client as SDSClient
from obspy.clients.fdsn import RoutingClient, Client
from obspy.core import UTCDateTime as UTC
from obspy.signal import util

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from data_quality_control import sds_db, base

In [ ]:
from obspy.clients import fdsn

## Define parameters

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'BHZ'

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

outdir = 'output'

sds_root = os.path.abspath('../../sample_sds/')
inventory_routing_type = "eida-routing"

In [ ]:
startdate = UTC("2020-12-24")
enddate = UTC("2021-01-15")

# Client testing

You can use the Clients directly to check the content
of the database. Note though, that this can take some time
if your database is large.

In [ ]:
sdsclient = SDSClient(sds_root)
invclient = RoutingClient(inventory_routing_type)

sdsclient.get_all_nslc()

In [ ]:
inv = invclient.get_stations(network=network, station="BFO", location=location, channel=channel,
                      starttime=startdate, endtime=enddate, level="response")

In [ ]:
inv

In [ ]:
inv_bgr = invclient.get_stations(network=network, station="BFO", location=location, channel=channel,
                      starttime=startdate, endtime=enddate, level="response")
inv_bgr

In [ ]:
inv = invclient.get_stations(network="CZ", station="NKC", location=location, channel=channel,
                      starttime=startdate, endtime=enddate, level="response")

In [ ]:
inv

In [ ]:
inv.

In [ ]:
network

In [ ]:
client = Client('http://eida.bgr.de', debug=True, timeout=300)

In [ ]:
invclient.__class__

In [ ]:
client.__class__

In [ ]:
RoutingClient(inv)

In [ ]:
inv.get_stations()

In [ ]:
Client

In [ ]:
isinstance(client, RoutingClient)

In [ ]:
client = Client('LMU', debug=True)

# Process raw data

Use the `processing` module to extract amplitudes and
power spectral densities (PSD) from the raw seismic data.

In [ ]:
#reload(processing)
proc_params = base.ProcessingParameters(proclen_seconds=proclen)
sdsp = sds_db.SDSProcessor("{}.{}.{}.{}".format(network, station, location, channel),
                                    inventory_routing_type, sds_root, 
                                        outdir=outdir, **proc_params.get_dict())

In [ ]:
sdsp.process(startdate, enddate)

In [ ]:
DEBUG - 2020-12-24T22:59:00.000000Z - 2020-12-25T00:01:00.000000Z

In [ ]:
reload(processing)
processor = processing.RawDataProcessor(
        network=network, 
        station=station,
        channel=channel,
        sds_root=sds_root,
        inventory_routing_type=inventory_routing_type,)

In [ ]:
%%time
#it -n1 -r7
processor.process(startdate, enddate, outdir,
                        overlap=overlap, 
            winlen_in_s=winlen_in_s,
        nperseg=nperseg, proclen=proclen,
                 amplitude_frequencies=(fmin, fmax))

## View processed data

With the `analysis` module, the processed data can be accessed 
and visualized.

In [ ]:
from data_quality_control import analysis
import numpy as np

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
startdate = UTC("2020-12-28")
enddate = UTC("2021-01-15")

In [ ]:
stationcode = "{}.{}.{}.{}".format(network, station, 
                                   location, channel)

In [ ]:
reload(analysis)
analyzer = analysis.Analyzer(startdate, enddate, outdir, stationcode)

In [ ]:
adata = analyzer.get_data(["amplitudes", "psds"], 
                         stime="00:00", etime="23:00"
                        #stime="19:00", etime="05:00"
                         )

In [ ]:
fig = analyzer.plot_amplitudes()

In a notebook or script you could simply run:

```python
fig.show()
```

In [ ]:
display(HTML(fig.to_html()))

In [ ]:
fig_psd = analyzer.plot_psds(np.log)

In [ ]:
display(HTML(fig_psd.to_html()))